In [2]:
%%sh
# this is new and  not shown in the book (--no-sign-request)
aws s3 cp --no-sign-request s3://amazon-reviews-pds/tsv/amazon_reviews_us_Camera_v1_00.tsv.gz /tmp

download: s3://amazon-reviews-pds/tsv/amazon_reviews_us_Camera_v1_00.tsv.gz to ../../../tmp/amazon_reviews_us_Camera_v1_00.tsv.gz


In [2]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv('/tmp/amazon_reviews_us_Camera_v1_00.tsv.gz', sep='\t', 
                   compression='gzip', error_bad_lines=False, dtype='str')
data.dropna(inplace=True)

b'Skipping line 85458: expected 15 fields, saw 22\nSkipping line 91161: expected 15 fields, saw 22\n'
b'Skipping line 166123: expected 15 fields, saw 22\n'
b'Skipping line 225458: expected 15 fields, saw 22\nSkipping line 229936: expected 15 fields, saw 22\nSkipping line 259297: expected 15 fields, saw 22\n'
b'Skipping line 284728: expected 15 fields, saw 22\nSkipping line 286334: expected 15 fields, saw 22\nSkipping line 293400: expected 15 fields, saw 22\nSkipping line 294415: expected 15 fields, saw 22\nSkipping line 308150: expected 15 fields, saw 22\nSkipping line 315022: expected 15 fields, saw 22\nSkipping line 315730: expected 15 fields, saw 22\nSkipping line 316071: expected 15 fields, saw 22\nSkipping line 326729: expected 15 fields, saw 22\n'
b'Skipping line 329101: expected 15 fields, saw 22\nSkipping line 333077: expected 15 fields, saw 22\nSkipping line 377031: expected 15 fields, saw 22\nSkipping line 389496: expected 15 fields, saw 22\nSkipping line 390486: expected 15 

In [4]:
print(data.shape)
print(data.columns)

(1800755, 15)
Index(['marketplace', 'customer_id', 'review_id', 'product_id',
       'product_parent', 'product_title', 'product_category', 'star_rating',
       'helpful_votes', 'total_votes', 'vine', 'verified_purchase',
       'review_headline', 'review_body', 'review_date'],
      dtype='object')


In [ ]:
data.head()

In [5]:
data = data.iloc[:100000]

In [6]:
data = data[['star_rating', 'review_body']]

In [7]:
data.star_rating.unique()

array(['5', '2', '3', '1', '4'], dtype=object)

In [8]:
data['label'] = data.star_rating.map({
    '1': '__label__negative__',
    '2': '__label__negative__',
    '3': '__label__neutral__',
    '4': '__label__positive__',
    '5': '__label__positive__'})

In [9]:
data = data.drop(['star_rating'], axis=1)

In [10]:
data = data[['label', 'review_body']]

In [11]:
data.head()

,label,review_body
0,__label__positive__,ok
1,__label__positive__,"Perfect, even sturdier than the original!"
2,__label__negative__,"If the words, &#34;Cheap Chinese Junk&#34; com..."
3,__label__positive__,Exactly what I wanted and expected. Perfect fo...
4,__label__positive__,I will look past the fact that they tricked me...


In [12]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [17]:
data['review_body'] = data['review_body'].apply(nltk.word_tokenize)

KeyboardInterrupt: 

In [18]:
# we can also tokenize using built python functions and methods.
# but it will be time taken, and may cause error
sample = "Perfect, even sturdier than the original!"
sample = sample.split(" ")
print(sample)
sample2 = data['review_body'].iloc[1]
print(sample2)
# it means nltk.tokernizer is better to use

['Perfect,', 'even', 'sturdier', 'than', 'the', 'original!']
perfect , even sturdier than the original !


In [16]:
%%time
data['review_body'] = data.apply(lambda row: " ".join(row['review_body']).lower(), axis=1)

CPU times: user 1.6 s, sys: 19.9 ms, total: 1.62 s
Wall time: 1.63 s


In [20]:
sample2 = data['review_body'].iloc[2]
print(sample2)

if the words , & # 34 ; cheap chinese junk & # 34 ; come to your mind when you see this , then congratulate yourself . you 're pretty close . one of the most important features of a 'security camera & # 34 ; is the ability to detect motion and record , especially when running on battery and limited storage space . i tested the motion detect on this camera in a few different environments so far ( i.e . low light , indoors , outdoors , etc.. ) and all i got was a sd card full of video triggers . all the testing was done with the low motion sensitivity setting . i ca n't even imagine what the high one would be. < br / > as others said , the low light operation is poor . i did one of my tests outdoors as the sun was setting . it was good until a point - then the camera started having seizures..it got fuzzy , went to black and white ... back to color ... then ultimately stabilized with lines across the screen. < br / > another oddity i noticed was that the firmware reported in the windows c

In [21]:
from sklearn.model_selection import train_test_split

training, validation = train_test_split(data, test_size=0.05)

In [22]:
print(training.shape)
print(validation.shape)

(95000, 2)
(5000, 2)


In [23]:
np.savetxt('/tmp/training.txt', training.values, fmt='%s')
np.savetxt('/tmp/validation.txt', validation.values, fmt='%s')

In [24]:
!head -2 /tmp/training.txt

__label__positive__ great product for great price . definitely worth the money .
__label__positive__ very smooth gearing - and mounts nicely on tripod .


In [25]:
# training prepration
import boto3
import sagemaker

print(sagemaker.__version__)

session = sagemaker.Session()
bucket = session.default_bucket()

2.31.1


In [ ]:
# Run this cell if you want to use the data you processed manually

prefix = 'amazon-reviews'

s3_train_path = session.upload_data(path='/tmp/training.txt', bucket=bucket, key_prefix=prefix+'/input/train')
s3_val_path = session.upload_data(path='/tmp/validation.txt', bucket=bucket, key_prefix=prefix+'/input/validation')
s3_output = 's3://{}/{}/output/'.format(bucket, prefix)

print(s3_train_path)
print(s3_val_path)
print(s3_output)

In [27]:
from sagemaker import image_uris

region = boto3.Session().region_name    
container = image_uris.retrieve('blazingtext', region)
print(container)

811284229777.dkr.ecr.us-east-1.amazonaws.com/blazingtext:1


In [31]:
role = sagemaker.get_execution_role()

bt = sagemaker.estimator.Estimator(container,
                                   role, 
                                   instance_count=1, 
                                   instance_type='ml.c5.2xlarge',
                                   output_path=s3_output)

In [33]:
bt.set_hyperparameters(mode='supervised')

In [34]:
train_data = sagemaker.TrainingInput(s3_train_path, 
                      distribution='FullyReplicated', 
                      content_type='text/plain',
                      s3_data_type='S3Prefix')

validation_data = sagemaker.TrainingInput(s3_val_path,
                      distribution='FullyReplicated', 
                      content_type='text/plain', 
                      s3_data_type='S3Prefix')

s3_channels = {'train': train_data, 'validation': validation_data}

In [35]:
bt.fit(inputs=s3_channels)

2021-04-25 11:33:22 Starting - Starting the training job...
2021-04-25 11:33:45 Starting - Launching requested ML instancesProfilerReport-1619350402: InProgress
......
2021-04-25 11:34:52 Starting - Preparing the instances for training.........
2021-04-25 11:36:17 Downloading - Downloading input data
2021-04-25 11:36:17 Training - Training image download completed. Training in progress.Arguments: train
[04/25/2021 11:36:18 WARNING 140665554687616] Loggers have already been setup.
[04/25/2021 11:36:18 WARNING 140665554687616] Loggers have already been setup.
[04/25/2021 11:36:18 INFO 140665554687616] nvidia-smi took: 0.025143146514892578 secs to identify 0 gpus
[04/25/2021 11:36:18 INFO 140665554687616] Running single machine CPU BlazingText training using supervised mode.
Number of CPU sockets found in instance is  1
[04/25/2021 11:36:18 INFO 140665554687616] Processing /opt/ml/input/data/train/training.txt . File size: 25.362000465393066 MB
[04/25/2021 11:36:18 INFO 140665554687616] P

In [36]:
bt_predictor = bt.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

-------------------!

In [39]:
import json
#import pprint

sentences = ['This is a bad camera it doesnt work at all , i want a refund  .', 
             'The camera works , the pictures are decent quality, nothing special to say about it .',
             'Very happy to have bought this , exactly what I needed']

payload = {"instances" : sentences, "configuration": {"k": 3}}

bt_predictor.serializer = sagemaker.serializers.JSONSerializer()
response = bt_predictor.predict(payload)

print(response)

b'[{"label": ["__label__negative__", "__label__neutral__", "__label__positive__"], "prob": [0.9729471802711487, 0.026321984827518463, 0.0007608418236486614]}, {"label": ["__label__neutral__", "__label__positive__", "__label__negative__"], "prob": [0.555188775062561, 0.30684155225753784, 0.13799965381622314]}, {"label": ["__label__positive__", "__label__neutral__", "__label__negative__"], "prob": [0.999725878238678, 0.0002516448439564556, 5.2410414355108514e-05]}]'


In [40]:
# codes not available in the book
# parsing the response
print(type(response))

<class 'bytes'>


In [46]:

data = response.decode("UTF-8")
print(type(data) ,data)

<class 'str'> [{"label": ["__label__negative__", "__label__neutral__", "__label__positive__"], "prob": [0.9729471802711487, 0.026321984827518463, 0.0007608418236486614]}, {"label": ["__label__neutral__", "__label__positive__", "__label__negative__"], "prob": [0.555188775062561, 0.30684155225753784, 0.13799965381622314]}, {"label": ["__label__positive__", "__label__neutral__", "__label__negative__"], "prob": [0.999725878238678, 0.0002516448439564556, 5.2410414355108514e-05]}]


In [47]:
import ast
mydata = ast.literal_eval(data) 
print(type(mydata), mydata)


<class 'list'> [{'label': ['__label__negative__', '__label__neutral__', '__label__positive__'], 'prob': [0.9729471802711487, 0.026321984827518463, 0.0007608418236486614]}, {'label': ['__label__neutral__', '__label__positive__', '__label__negative__'], 'prob': [0.555188775062561, 0.30684155225753784, 0.13799965381622314]}, {'label': ['__label__positive__', '__label__neutral__', '__label__negative__'], 'prob': [0.999725878238678, 0.0002516448439564556, 5.2410414355108514e-05]}]


In [64]:
import ast
def reviewPrediction(response):
    data = response.decode("UTF-8")
    myData = ast.literal_eval(data) # oonversting a string (with list and dictionary marker into list or dictionary)
    for line in myData:
        #print(line, len(line))
        label = line['label']
        prob = line['prob']
        #print(label)
        #print(prob)
        # np.argmax is not required here, as highest probalities showing at 0 position,
        # but for a good practice, we are using np.argmax  
        position = np.argmax(prob)
        print(label[position])

In [65]:
results = reviewPrediction(response)
print(results)

__label__negative__
__label__neutral__
__label__positive__
None


In [79]:
sentences = [' i am  very disappointed . its all rubbish', ' very very bad']

payload = {"instances" : sentences, "configuration": {"k": 3}}

bt_predictor.serializer = sagemaker.serializers.JSONSerializer()
response = bt_predictor.predict(payload)

results = reviewPrediction(response)
print(results)

__label__negative__
__label__negative__
None


In [80]:
bt_predictor.delete_endpoint()